In [ ]:
# run against production 2024-10-10
from acdh_tei_pyutils.tei import TeiReader
from normdata.utils import import_from_normdata
from tqdm import tqdm

In [ ]:
doc = TeiReader("https://schubert-digital.at/export/personen.xml")

In [ ]:
nsmap = {"tei": "http://www.music-encoding.org/ns/mei"}
col, _ = Collection.objects.get_or_create(name="Schubert Digital")
domain = "schubert-digital"

In [ ]:
broken_gnd = []
pmb_uris = []
for x in tqdm(doc.tree.xpath(".//tei:persName[@xml:id]", namespaces=nsmap)):
    try:
        gnd = x.xpath(".//tei:identifier[@auth='GND']", namespaces=nsmap)[0].text
    except:
        gnd = False
        continue
    if gnd:
        xml_id = x.xpath(".//tei:identifier[@label='Schubert-Personenregister-ID']/text()", namespaces=nsmap)[0]
        domain_uri = f"https://schubert-digital.at/{xml_id}.html"
        gnd_uri = f"https://d-nb.info/gnd/{gnd}"
        entity = import_from_normdata(gnd_uri, "person")
        if entity:
            pmb_uri, _ = Uri.objects.get_or_create(uri=domain_uri, domain=domain)
            pmb_uri.entity = entity
            pmb_uri.save()
            entity.collection.add(col)
            pmb_uris.append([xml_id, entity.id])
        else:
            broken_gnd.append(gnd_uri)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(pmb_uris)

In [ ]:
df.to_csv('schubert_pmb.csv', index=False)